# Rotten Tomatoes Sentiment Prediction

## Exploration
- pattern sentiment (but minimize)
- subj/obj and phrase length
- modality and mood (summarize all the pattern stuff bc we didn't really use it)
- figure out what word2vec does - if it's similar to nltk tagging then include that

- sentence sentiment vs average sentiment of its phrases

- WORD CLOUD
- include most pos/neg words (relate to naive bayes + flow into scikit learn)

## Classification Models
- copy the model-iterations notebook but clean the code to make sure it isn't repetitive

## Neural Networks
- Code from lasagne and results
- add lots of comments to show we know what it does